# Section 1 Exercises

## Setup

### Using this notebook with ollama: 

- Install ollama
- Pull the `llama3.1` model by running `ollama pull llama3.1`
- Create `.env` and write

```ini
MODE=ollama
```

- Restart the kernel

### Using this notebook with GitHub Models: 

- Create `.env` and write

```ini
MODE=github
```

- Go to [Access Tokens](https://github.com/settings/personal-access-tokens)
- Create new **Fine-tuned token**
- Set timeout to be some time in the future (how long you plan on using it)
- Expand Account Permissions
- Find **Models** and change to read-only
- Save
- Copy the key
- Add it to `.env` as `GITHUB_TOKEN=gha....`
- Restart the kernel


In [ ]:
from IPython.display import display, Markdown


In [ ]:
# run this cell to install the required packages, I suggest doing this first. 
%pip install -r requirements.txt


# OpenAI Byte-Pair-Encodings

Many AI models use OpenAI's Byte-Pair-Encodings

- `cl100k_base` (GPT3.5 to GPT4)
- `o200k_base` (GPT-4o, o1, o3, GPT-4.1)

The easiest and fastest way to use these encodings is with the `tiktoken` library.

In [ ]:
# Tokenization for OpenAI models

from tiktoken import encoding_for_model, get_encoding

# Load the encoding for a known OpenAI model
encoding = encoding_for_model("gpt-3.5-turbo")
# or use a specific encoding
# encoding = get_encoding("cl100k_base")

# Encode the text
text = "Hello, world!"
tokens = encoding.encode(text)

tokens

In [ ]:
# Convert tokens back to text
decoded_text = encoding.decode(tokens)

decoded_text

# Hugging Face Tokenizers

Many open models use custom tokenizers and byte-pair-encodings.

HuggingFace Models have an API for storing and downloading tokenizers to process inputs locally.

The `tokenizers` package on PyPi facilitates this, but the `transformers` package makes it even easier to download and use them.

Use the `AutoTokenizer` class to download and install the tokenizer for a given model.

You will need to create an account on hugging face and login using:

```bash
huggingface_hub login
```


In [ ]:
# Using huggingface tokenizers
from transformers import AutoTokenizer

# Load the tokenizer for a specific model
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B")

# Encode the text
tokens = tokenizer(text, 
                   add_special_tokens=False, # End of stream/end of text tokens
                   return_offsets_mapping=False) # Return offsets for each token

tokens

In [ ]:
# Alternatively, you can use the tokenizer directly
tokens = tokenizer.encode(text)
tokens

In [ ]:
# Decode the tokens back to text
decoded_text = tokenizer.decode(tokens)

decoded_text

In [ ]:
# Input text doesn't need to be English or ASCII
text = "你好，世界！"
tokens = encoding.encode(text)
decoded_text = encoding.decode(tokens)

tokens, decoded_text

In [ ]:
# Although, the number of tokens may be different
# depending on the encoding used
text = "你好，世界！"

encoding_gpt2 = get_encoding("gpt2") # Old, but used in some open models
encoding_cl100 = get_encoding("cl100k_base") # GPT3.5 - 4
encoding_o200 = get_encoding("o200k_base") # New encoding. Double the size of cl100k_base and optimized for languages like Chinese and Japanese

print("GPT-2 is ", len(encoding_gpt2.encode(text)), "tokens")
print("CL100k_base is ", len(encoding_cl100.encode(text)), "tokens")
print("and o200 is", len(encoding_o200.encode(text)), "tokens")

# Using OpenAI client for chat completions

The OpenAI Python client has become the unofficial standard and most models are work with OpenAI's API spec.

So, even if you're not using OpenAI you can still use the Python client to talk to GitHub models, Ollama local models and many 3rd party LLMs.

Normally, you install it with `pip install openai` but it's already a requirement for this notebook.

In [ ]:
# Your first conversation with a model
from openai import OpenAI
import utils

# If you change the environment variables, you need to restart the kernel
base_url = utils.get_base_url()
api_key = utils.get_api_key()

if utils.MODE == "github":
    model = "openai/gpt-4.1-nano"  # A fast, small model
elif utils.MODE == "ollama":
    model = "llama3.1"  # llama and ollama are not related. It's a coincidence

# OpenAI client is a class. The old API used to use globals. Sometimes you might see code snippets for the old API. 

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of France?",
        }
    ],
    temperature=0.5,  # or top_p=0.9
    n=1, # Number of results to return. If you want multiple options, increase this

    # Here are some extra parameters you might need in future

    # presence_penalty=0.0, # Increase the likelihood of new topics, default is 0. Range is -1 to 1
    # frequency_penalty=0.0, # Increase the likelihood of new words, default is 0. Range is -2 to 2
    # max_tokens=100, # Maximum number of tokens to return
    # stop=None, # Stop when the model generates this token
)

display(Markdown(response.choices[0].message.content))

# Exercise 1: Adjust the temperature and see how it affects the output

Copy the code above and ask it to create a **poem about the moon**
Try different values for temperature and see how it affects the output
Try varying frequency_penalty and presence_penalty too

In [ ]:
# Create your moon poem

Below we have written a grading system using another LLM. You will get a grade from A-F along with the reasoning. See if you can adjust your prompt to get a better grade.

In [ ]:
# Put a better poem here
poem = """
Oh moonie moonie, shining bright,
I love to see you in the night.
You light the way for all to see,
And fill my heart with joy and glee.
"""

grade = utils.grade_poem(client, model, poem)


display(Markdown(grade))

# Zero, one and few shot prompts

## Game

We're going to test an AI by giving it a made-up card game and getting it to work out who the winner is.


In [ ]:
from utils.game import RULES

display(Markdown(RULES))

In [ ]:
# Zero, one, and few-shot prompting
from utils.game import determine_winner_programmatically


instructions = f"""You are an AI for a fictional card game. Given the players and the cards they play, you need to determine the winner of the game. Return only the number of the player.
The rules of the game are:

{RULES}
"""

def determine_winner(turn):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": instructions,
            },
            {
                "role": "user",
                "content": turn,
            }
        ],
        temperature=1.0,
        n=1, # Number of results to return. If you want multiple options, increase this
    )

    result = response.choices[0].message.content
    return result

# Example turn
test_turn = {
    "Player 1": "2 of hearts",
    "Player 2": "3 of diamonds",
    "Player 3": "4 of diamonds"
}

turn = f"""
Player 1: {test_turn['Player 1']}
Player 2: {test_turn['Player 2']}
Player 3: {test_turn['Player 3']}
"""
ai_result = determine_winner(turn)
actual_winner = determine_winner_programmatically(test_turn)

if ai_result == actual_winner:
    display(Markdown(f"AI result: {ai_result} is correct!"))
else:
    display(Markdown(f"AI result: {ai_result} is incorrect! Actual result: {actual_winner}"))



In [ ]:
# Task 1: Introduce one-shot by extending the `instructions` and including an example turn and result. 

In [ ]:
# Task 2: Introduce few-shot by extending the messages with example turns and results

"""
e.g. 

messages=[
    {
        "role": "system",
        "content": instructions,
    },
    {
        "role": "user",
        "content": "Player 1: 2 of hearts\nPlayer 2: 3 of spades\nPlayer 3: 4 of diamonds",
    },
    {
        "role": "assistant",
        "content": f"Player 2 wins!",
    },
    {
        "role": "user",
        "content": turn,
    },

]

"""

In [ ]:
# Finally, test out the AI with a few turns
from utils.game import create_training_data

total_score = 0

for plays, winner in create_training_data(10):
    turn = f"""
    Player 1: {test_turn['Player 1']}
    Player 2: {test_turn['Player 2']}
    Player 3: {test_turn['Player 3']}
    """
    ai_result = determine_winner(turn)
    actual_winner = winner

    if ai_result == actual_winner:
        display(Markdown(f"AI result: {ai_result} is correct!"))
        total_score += 1
    else:
        display(Markdown(f"AI result: {ai_result} is incorrect! Actual result: {actual_winner}"))

display(Markdown(f"Your score is {total_score} out of 10"))


In [ ]:
# Bonus task.

# If you finish early, try and experiment with what happens when you change the number of players.